In [ ]:
from lec_utils import *
def show_grouping_animation():
    src = "https://docs.google.com/presentation/d/1tBaFyHseIGsX5wmE3BdNLeVHnKksQtpzLhHge8Tzly0/embed?start=false&loop=false&delayms=60000&rm=minimal"
    width = 960
    height = 509
    display(IFrame(src, width, height))

<div class="alert alert-info" markdown="1">

#### Lecture 5

# Aggregation: Grouping and Pivoting

### EECS 398: Practical Data Science, Winter 2025

<small><a style="text-decoration: none" href="https://practicaldsc.org">practicaldsc.org</a> • <a style="text-decoration: none" href="https://github.com/practicaldsc/wn25">github.com/practicaldsc/wn25</a> • 📣 See latest announcements [**here on Ed**](https://edstem.org/us/courses/69737/discussion/5943734) </small>
    
</div>

### Agenda 📆

- Introduction to the `groupby` method.
- `groupby`'s inner workings.
- Advanced `groupby` usage.
- Pivot tables using `pivot_table`.

Remember to follow along in lecture by accessing the "blank" lecture notebook in our [public GitHub repository](https://github.com/practicaldsc/wn25).

<div class="alert alert-success">
    
### Read the guide!
    
- We've posted a new guide about how DataFrames are stored in memory, linked [**here**](https://practicaldsc.org/guides/data-wrangling/df-internals/).
- It covers:
    - How to (correctly) add new columns to DataFrames, and otherwise deal with the fact that they are **mutable**.
    - How DataFrames interface with `numpy`.
- This content used to be part of lecture last semester, but we've moved it into a guide to spend lecture time on more conceptual material.
- **But you're still responsible for knowing it!**
    
</div>

<div class="alert alert-warning">
    <h3>Question 🤔 (Answer at <a style="text-decoration: none; color: #0066cc" href="https://docs.google.com/forms/d/e/1FAIpQLSd4oliiZYeNh76jWy-arfEtoAkCrVSsobZxPwxifWggo3EO0Q/viewform">practicaldsc.org/q</a>)</h3>
    
<small>Remember that you can always ask questions anonymously at the link above!</small>

## Introduction to the `groupby` method

---

What sorts of problems does `groupby` help us solve?

### Example: Palmer Penguins

<center><img src="imgs/lter_penguins.png" width=60%>
<i><a href="https://github.com/allisonhorst/palmerpenguins/blob/main/README.md">Artwork by @allison_horst</a></i>

</center>

The dataset we'll work with for the rest of the lecture involves various measurements taken of three species of penguins in Antarctica.

In [ ]:
IFrame('https://www.youtube-nocookie.com/embed/CCrNAHXUstU?si=-DntSyUNp5Kwitjm&amp;start=11',
       width=560, height=315)

### Loading the data

In [ ]:
penguins = pd.read_csv('data/penguins.csv')
penguins

- Here, each row corresponds to a single penguin, and each column corresponds to a different attribute (or feature) we have for each penguin.

- Data formatted in this way is sometimes called [tidy data](https://r4ds.had.co.nz/tidy-data.html).

### Visualizing the data

In [ ]:
penguins.plot(kind='scatter', 
              x='bill_length_mm', 
              y='body_mass_g', 
              color='species', 
              title='Body Mass vs. Bill Length')

### Aggregating

- **Aggregating** is the act of combining many values into a single value.

- Aggregations "hide" some of the detail in the data, and help understand bigger-picture trends.

- Example: What is the mean `'body_mass_g'` for all penguins?

In [ ]:
...

- Example: What is the mean `'body_mass_g'` **for each `'species'`**?

### A naïve approach to finding the mean `'body_mass_g'` per `'species'`

- First, we could identify all unique values in the `'species'` column.

In [ ]:
...

- Then, for each `'species'`, we could:
    1. **Query** for just that `'species'`.
    1. Extract the `'body_mass_g'` column and use the `mean` method on it.

In [ ]:
...

In [ ]:
...

In [ ]:
...

- We _could_ use a `for`-loop, but remember, we want to avoid Python `for`-loops.

### The magic of `groupby` 🪄

- A better solution is to use the `groupby` method.

In [ ]:
# To find the overall mean 'body_mass_g':
...

In [ ]:
# To find the mean 'body_mass_g' for each 'species':
...

- Somehow, the `groupby` method computes what we're looking for in just one line. How?

- We'll work through the internals, but remember this: **if you need to calculate something _for each group_, use `groupby`!**

### An illustrative example: Pets 🐱 🐶🐹

- Consider the DataFrame `pets`, shown below.

<table border="1" class="dataframe">
  <thead>
    <tr style="text-align: right;">
      <th></th>
      <th>Species</th>
      <th>Color</th>
      <th>Weight</th>
      <th>Age</th>
    </tr>
  </thead>
  <tbody>
    <tr>
      <th>0</th>
      <td>dog</td>
      <td>black</td>
      <td>40</td>
      <td>5.0</td>
    </tr>
    <tr>
      <th>1</th>
      <td>cat</td>
      <td>golden</td>
      <td>15</td>
      <td>8.0</td>
    </tr>
    <tr>
      <th>2</th>
      <td>cat</td>
      <td>black</td>
      <td>20</td>
      <td>9.0</td>
    </tr>
    <tr>
      <th>3</th>
      <td>dog</td>
      <td>white</td>
      <td>80</td>
      <td>2.0</td>
    </tr>
    <tr>
      <th>4</th>
      <td>dog</td>
      <td>golden</td>
      <td>25</td>
      <td>0.5</td>
    </tr>
    <tr>
      <th>5</th>
      <td>hamster</td>
      <td>golden</td>
      <td>1</td>
      <td>3.0</td>
    </tr>
  </tbody>
</table>

- Let's see what happens under the hood when we use the `groupby` method on `pets`.

In [ ]:
show_grouping_animation()

### "Split-apply-combine" paradigm

- The `groupby` method involves three steps: **split**, **apply**, and **combine**.<br><small>This is the same terminology that the [`pandas` documentation](https://pandas.pydata.org/docs/user_guide/groupby.html) uses.</small>

<center><img src="imgs/image_0.png" width=700></center>

- **Split** breaks up and "groups" the rows of a DataFrame according to the specified **key**.<br><small>There is one "group" for every unique value of the key.</small>

- **Apply** uses a function (e.g. aggregation, transformation, filtration) within the individual groups.

- **Combine** stitches the results of these operations into an output DataFrame.

- The split-apply-combine pattern can be **parallelized** to work on multiple computers or threads, by sending computations for each group to different processors.

<div class="alert alert-success">
<h3>Activity</h3>
        
Which penguin `'species'` has the highest median `'bill_length_mm'`?

In [ ]:
penguins

In [ ]:
...

In [ ]:
...

## `groupby`'s inner workings

---

### How does `groupby` actually work?

- We've just evaluated a few expressions of the following form.

In [ ]:
penguins.groupby('species')['bill_length_mm'].mean()

- There are three "building blocks"
in the above expression:
    1. `penguins.groupby('species')`.<br><small>First, we specify which column we want to group on.</small>
    1. `['bill_length_mm']`.<br><small>Then, we select the other relevant columns for our calculations.</small>
    1. `.mean()`.<br><small>Finally, we use an aggregation method.</small>

- Let's see what each block contributes to the output.

### `DataFrameGroupBy` objects

<pre>

<center><strong><span style="color: #0066cc">penguins.groupby('species')</span></strong><span style="color: #999">['bill_length_mm'].mean()</span></center>

</pre>

- If `df` is a DataFrame, then `df.groupby(key)` returns a `DataFrameGroupBy` object.<br><small>This object represents the "split" in "split-apply-combine".</small>

In [ ]:
penguins.groupby('species')

In [ ]:
# Creates one group for each unique value in the species column.
penguins.groupby('species').groups

- `DataFrameGroupBy` objects have a `groups` attribute, which is a dictionary in which the keys are group names and the values are lists of row labels.<br><small>We won't actually use this, but it's helpful in understanding how `groupby` works under-the-hood.</small>

### Column extraction

<pre>

<center><span style="color: #999">penguins.groupby('species')</span><strong><span style="color: #0066cc">['bill_length_mm']</span></strong><span style="color: #999">.mean()</span></center>

</pre>

- After creating a `DataFrameGroupBy` object, we typically select the relevant column(s) that we want to aggregate.<br><small>If we don't, we may run into errors if trying to use a numeric aggregation method on non-numeric columns, as we saw earlier.<br>Also, this is more efficient, since the aggregations are only performed on the columns you care about, rather than all columns.</small>

- The result is either a `SeriesGroupBy` or `DataFrameGroupBy` object, depending on what's passed in.

In [ ]:
...

In [ ]:
...

### Aggregation

<pre>

<center><span style="color: #999">penguins.groupby('species')['bill_length_mm']</span><strong><span style="color: #0066cc">.mean()</span></strong></center>

</pre>

- Once we create a `DataFrameGroupBy` or `SeriesGroupBy` object, we need to **apply** some function **separately** to each group, and **combine** the results.

- The most common operation we apply to each group is an **aggregation**, but we'll see examples of **filtrations** and **transformations** soon.<br><small>Remember, aggregation is the act of combining many values into a single value.</small>

- To perform an aggregation, use an aggregation method on the `DataFrameGroupBy` or `SeriesGroupBy` object, e.g. `.mean()`, `.max()`, or `.median()`.

Let's look at some examples.

In [ ]:
# Note that this worked on the entire DataFrame!
# But, if all we wanted are the sums of `'body_mass_g'
# for each species, this is slower than
# penguins.groupby('species')['body_mass_g'].sum().
...

In [ ]:
# Often used in conjunction with sort_values.
# Remember this when you work on the activity in a few slides!
...

In [ ]:
# Similar to value_counts, but not identical!
...

In [ ]:
...

### Reminder: Column independence

- As we've seen, within each group, the aggregation method is applied to **each column independently**.

In [ ]:
penguins.groupby('species').max()

- The above result **is not** telling us that there is a `'Adelie'` penguin with a `'body_mass_g'` of `4775.0` that lived on `'Torgersen'` island.

In [ ]:
# This penguin lived on Biscoe island!
penguins.loc[(penguins['species'] == 'Adelie') & (penguins['body_mass_g'] == 4775.0)]

<div class="alert alert-success">
<h3>Activity</h3>

Find the <code>'species'</code>, <code>'island'</code>, and <code>'body_mass_g'</code> of the heaviest <code>'Male'</code> and <code>'Female'</code> penguins in <code>penguins</code> (not <code>penguins_small</code>).
</div>

In [ ]:
...

<div class="alert alert-success">
<h3>Activity</h3>
                
What proportion of penguins of each `'species'` live on `'Dream'` island?
    
***Hint***: If you've read the guide, this activity will be much easier!

</div>

In [ ]:
...

## Advanced `groupby` usage

---

### Beyond default aggregation methods

- There are many built-in aggregation methods, like `.mean()`, `.max()`, and `.last()`.

- What if the aggregation method you want to use doesn't already exist in `pandas`, or what if you want to apply different aggregation methods to different columns?

- Or, what if you don't want to perform an aggregation, but want to perform **some other operation** separately on each group?

### Grouping method 1:  `agg`

- After grouping, use the `agg` method if you want to aggregate with:
    - A single function (either built-in or one that you define!).
    - A list of functions.
    - A dictionary mapping column names to functions.
    
    Refer to [the documentation](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.core.groupby.DataFrameGroupBy.aggregate.html) for a comprehensive list.<br><small>Per [the documentation](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.agg.html), `agg` is an alias for `aggregate`.</small>

- Example: How many penguins are there of each `'species'`, and what is the mean `'body_mass_g'` of each `'species'`?

In [ ]:
(
    penguins
    .groupby('species')
    ['body_mass_g']
    .agg(['count', 'mean'])
)

- Note that we call `agg` a "grouping method" because it comes after a call to `groupby`.

### `agg` with different aggregation methods for different columns

- Example: What is the maximum `'bill_length_mm'` of each `'species'`, and which `'island'`s is each `'species'` found on?

In [ ]:
(
    penguins
    .groupby('species')
    .aggregate({'bill_length_mm': 'max', 'island': 'unique'})
)

### `agg` with a custom aggregation method

- **Example**: What is the **second largest** recorded `'body_mass_g'` for each `'species'`?

In [ ]:
# Here, the argument to agg is a function,
# which takes in a Series and returns a scalar.
...

<div class="alert alert-info">

- **Key idea**: If you give `agg` a custom function, it should map $\texttt{Series} \rightarrow \texttt{number}$.
    
</div>

### Grouping method 2: `filter`

- As we saw last class, a **query** keeps **rows** that satisfy conditions.<br>For instance, to see the individual penguins with a `'bill_length_mm'` over 47 mm:

In [ ]:
# This is a query, NOT a filter for the purposes of this slide.
penguins[penguins['bill_length_mm'] > 47]

- A **filter**, on the other hand, keeps **entire groups** that satisfy conditions.

- For instance, to see the **penguin `'species'`** with an _average_ `'bill_length_mm'` over 47 mm, use the `filter` method after `groupby`:

In [ ]:
...

- Notice that the above DataFrame has 187 rows, fewer than the 333 in the full DataFrame. That's because there are **no `'Adelie'` penguins** above!

In [ ]:
# Since 'Adelie's have a mean 'bill_length_mm' below 47, they aren't included in the output above.
penguins.groupby('species')['bill_length_mm'].mean()

<div class="alert alert-info">

- **Key idea**: If you give `filter` a custom function, it should map $\texttt{DataFrame} \rightarrow \texttt{Boolean}$. The resulting DataFrame will only have the groups for which the custom function returned `True`.
    
</div>

<div class="alert alert-success">
<h3>Activity</h3>

There is only one penguins `'species'` with:
- At least 100 penguins.
- At least 60 `'Female'` penguins.
    
Find the `'species'` using **a single expression** (i.e. no intermediate variables). Use `filter`.

In [ ]:
...

In [ ]:
...

In [ ]:
...

<div class="alert alert-warning">
    <h3>Question 🤔 (Answer at <a style="text-decoration: none; color: #0066cc" href="https://docs.google.com/forms/d/e/1FAIpQLSd4oliiZYeNh76jWy-arfEtoAkCrVSsobZxPwxifWggo3EO0Q/viewform">practicaldsc.org/q</a>)</h3>
    
<small>Remember that you can always ask questions anonymously at the link above!</small>
    
What questions do you have?

### Grouping method 3: `transform`

- Use the `transform` grouping method if you want to apply a function separately to each group.

- **Example**: How much heavier is each penguin than the average penguin **of their `'species'`**?

In [ ]:
...

In [ ]:
...

- Notice that penguin 332's transformed `'body_mass_g'` is negative, even though their actual `'body_mass_g'` is very large; this is because they have a below-average `'body_mass_g'` for their `'species'`.

<div class="alert alert-info">

- **Key idea**: If you give `transform` a custom function, it should map $\texttt{Series} \rightarrow \texttt{Series}$.
    
</div>

### The `apply` grouping method

- Sometimes, you want to apply an operation separately for each group, but the operation isn't possible using `agg`, `filter`, or `transform`.

- The `apply` grouping method is like a swiss-army knife.<br>It can do anything `agg` or `transform` can do, and more.<br>**Only use it if necessary, because it's slower than `agg` and `transform`!**

- **Example**: Find the two heaviest penguins per `'species'`.

In [ ]:
...

- **Example**: Find the `'flipper_length_mm'` of the heaviest penguin of each `'species'`.

In [ ]:
...

<div class="alert alert-info">

- **Key idea**: If you give `agg` a custom function, it should map $\texttt{DataFrame} \rightarrow \texttt{anything}$. The outputs of the custom function will be stitched together intelligently by `pandas`.
    
</div>

### ⭐️ The grouping method cheat sheet: `agg`, `filter`, `transform`, and `apply` ⭐️

After grouping, use:

- The `agg` method if you want to aggregate values, separately for each group.<br><small>If you give `agg` a custom function, it should map $\texttt{Series} \rightarrow \texttt{number}$.</small>

In [ ]:
penguins.groupby('species')['body_mass_g'].agg(lambda s: s.sort_values().iloc[-2])

- The `filter` method if you want to keep groups that satisfy certain conditions.<br><small>If you give `filter` a custom function, it should map $\texttt{DataFrame} \rightarrow \texttt{Boolean}$. The resulting DataFrame will only have the groups for which the custom function returned `True`.</small>

In [ ]:
(
    penguins
    .groupby('species')
    .filter(lambda df: (df.shape[0] >= 100) and ((df['sex'] == 'Female').sum() >= 60))
)

- The `transform` method if you want to modify the values within each group separately.<br><small>If you give `transform` a custom function, it should map $\texttt{Series} \rightarrow \texttt{Series}$.

In [ ]:
penguins.groupby('species')['body_mass_g'].transform(lambda s: s - s.mean())

- The `apply` method if you want to perform some general operation on each group separately.<br><small>If you give `apply` a custom function, it should map $\texttt{DataFrame} \rightarrow \texttt{anything}$.

In [ ]:
penguins.groupby('species').apply(lambda df: df.sort_values('body_mass_g', ascending=False).head(2))

<div class="alert alert-warning">
    <h3>Question 🤔 (Answer at <a style="text-decoration: none; color: #0066cc" href="https://docs.google.com/forms/d/e/1FAIpQLSd4oliiZYeNh76jWy-arfEtoAkCrVSsobZxPwxifWggo3EO0Q/viewform">practicaldsc.org/q</a>)</h3>
    
<small>Remember that you can always ask questions anonymously at the link above!</small>
    
What questions do you have?

### Grouping with multiple columns

- When we group with multiple columns, one group is created for **every unique combination** of elements in the specified columns.<br><small>In the output below, why are there only 5 rows, rather than $3 \times 3 = 9$ rows, when there are 3 unique `'species'` and 3 unique `'island'`s?</small>

In [ ]:
# Read this as:
species_and_island = (
    penguins.groupby(['species', 'island'])         # for every combination of 'species' and 'island' in the DataFrame,
    [['bill_length_mm', 'bill_depth_mm']].mean()    # calculate the mean 'bill_length_mm' and the mean 'bill_depth_mm'.
)
species_and_island

- **Advice**: When grouping on multiple columns, the result usually has a `MultiIndex`;  use `reset_index` or set `as_index=False` in `groupby` to avoid this.

In [ ]:
# Now, this looks like a regular DataFrame!
...

## Pivot tables using `pivot_table`

---

### Pivot tables: An extension of grouping

- Pivot tables are a compact way to display tables for humans to read.

<table border="1" class="dataframe">
  <thead>
    <tr style="text-align: right;">
      <th>sex</th>
      <th>Female</th>
      <th>Male</th>
    </tr>
    <tr>
      <th>species</th>
      <th></th>
      <th></th>
    </tr>
  </thead>
  <tbody>
    <tr>
      <th>Adelie</th>
      <td>3368.84</td>
      <td>4043.49</td>
    </tr>
    <tr>
      <th>Chinstrap</th>
      <td>3527.21</td>
      <td>3938.97</td>
    </tr>
    <tr>
      <th>Gentoo</th>
      <td>4679.74</td>
      <td>5484.84</td>
    </tr>
  </tbody>
</table>


- Notice that each value in the table is the average of `'body_mass_g'` of penguins, for every combination of `'species'` and `'sex'`.

- **You can think of pivot tables as grouping using two columns, then "pivoting" one of the group labels into columns.**

### `pivot_table`

- The `pivot_table` DataFrame method aggregates a DataFrame using two columns. To use it:
<br><br>
```python
        df.pivot_table(index=index_col,
                       columns=columns_col,
                       values=values_col,
                       aggfunc=func)
```

- The resulting DataFrame will have:
    - One row for every unique value in `index_col`.
    - One column for every unique value in `columns_col`.
    - Values determined by applying `func` on values in `values_col`.

- **Example**: Find the average `'body_mass_g'` for every combination of `'species'` and `'sex'`.

In [ ]:
...

In [ ]:
...

### Example: Finding the number of penguins per `'island'` and `'species'`

In [ ]:
penguins

- Suppose we want to find the number of penguins in `penguins` per `'island'` and `'species'`. We can do so _without_ `pivot_table`:

In [ ]:
...

In [ ]:
...

- But the data is arguably easier to interpret when we do use `pivot_table`:

In [ ]:
...

- Note that there is a `NaN` at the intersection of `'Biscoe'` and `'Chinstrap'`, because there were no Chinstrap penguins on Biscoe Island.<br><small>`NaN` stands for "not a number." It is `numpy` and `pandas`' version of a null value (the regular Python null value is `None`). We'll learn more about how to deal with these soon.</small>

- We can either use the `fillna` method afterwards or the `fill_value` argument to fill in `NaN`s.

In [ ]:
...

### Granularity

- Each row of the original `penguins` DataFrame represented a single penguin, and each column represented features of the penguins.

In [ ]:
penguins

- What is the **granularity** of the DataFrame below?<br><small>That is, what does each row represent?</small>

In [ ]:
penguins.pivot_table(
    index='species', 
    columns='island', 
    values='bill_length_mm', 
    aggfunc='count',
    fill_value=0,
)

### Reshaping

- `pivot_table` reshapes DataFrames from "long" to "wide".

- Other DataFrame reshaping methods:
    - `melt`: Un-pivots a DataFrame. Very useful in data cleaning.
    - `pivot`: Like `pivot_table`, but doesn't do aggregation.
    - `stack`: Pivots multi-level columns to multi-indices.
    - `unstack`: Pivots multi-indices to columns.

- Google, the documentation, and ChatGPT are your friends!

<div class="alert alert-warning">
    <h3>Question 🤔 (Answer at <a style="text-decoration: none; color: #0066cc" href="https://docs.google.com/forms/d/e/1FAIpQLSd4oliiZYeNh76jWy-arfEtoAkCrVSsobZxPwxifWggo3EO0Q/viewform">practicaldsc.org/q</a>)</h3>
    
<small>Remember that you can always ask questions anonymously at the link above!</small>
    
What questions do you have?